In [1]:
import os
import pandas as pd
# Define the base path and the range of years
base_path = '../team/'
start_year = 2025
end_year = 2025
frames = []
# Loop through each year and list files in the corresponding directory
for year in range(start_year, end_year + 1):
    # Construct the path for the current year
    year_path = os.path.join(base_path, str(year))
    
    # Check if the directory exists
    if os.path.isdir(year_path):
        print(f"\nFiles in directory {year_path}:")
        
        # List all files in the directory
        for filename in os.listdir(year_path):
            file_path = os.path.join(year_path, filename)
            # Check if it is a file before printing
            if 'vs' not in filename and 'avg' not in filename:
                if '.csv' in filename:
                    
                    
                    df=pd.read_csv(file_path)
                    if len(df)>0:
                        #print(filename)
                        df['year']=year
                        frames.append(df)
        print(year)

    else:
        print(f"\nDirectory {year_path} does not exist.")

master=pd.concat(frames)
master


Files in directory ../team/2025:
2025


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SHOT_ID,time,extra,PLAYERS_ON,year
0,Shot Chart Detail,22400006,11,1626164,Devin Booker,1610612756,Phoenix Suns,1,11,11,...,1,1,20241112,UTA,PHX,2240000611,490,0,203078|203486|1626145|1626164|1642346,2025
1,Shot Chart Detail,22400006,14,1642346,Ryan Dunn,1610612756,Phoenix Suns,1,10,30,...,1,0,20241112,UTA,PHX,2240000614,900,0,203078|203486|1626145|1626164|1642346,2025
2,Shot Chart Detail,22400006,24,203078,Bradley Beal,1610612756,Phoenix Suns,1,9,44,...,1,0,20241112,UTA,PHX,2240000624,1360,0,203078|203486|1626145|1626164|1642346,2025
3,Shot Chart Detail,22400006,30,1626164,Devin Booker,1610612756,Phoenix Suns,1,8,42,...,1,1,20241112,UTA,PHX,2240000630,1980,0,203078|203486|1626145|1626164|1642346,2025
4,Shot Chart Detail,22400006,34,203078,Bradley Beal,1610612756,Phoenix Suns,1,8,10,...,1,1,20241112,UTA,PHX,2240000634,2300,0,203078|203486|1626145|1626164|1642346,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,Shot Chart Detail,22400301,620,1626162,Kelly Oubre Jr.,1610612755,Philadelphia 76ers,4,4,29,...,1,1,20241130,DET,PHI,22400301620,26110,0,1626162|1630178|1630231|1641741|1642272,2025
1553,Shot Chart Detail,22400301,650,1642272,Jared McCain,1610612755,Philadelphia 76ers,4,2,47,...,1,0,20241130,DET,PHI,22400301650,27130,0,1631311|1641737|1641741|1642272|1642348,2025
1554,Shot Chart Detail,22400301,658,1641741,Ricky Council IV,1610612755,Philadelphia 76ers,4,1,54,...,1,1,20241130,DET,PHI,22400301658,27660,0,1631311|1641737|1641741|1642272|1642348,2025
1555,Shot Chart Detail,22400301,662,1642272,Jared McCain,1610612755,Philadelphia 76ers,4,1,31,...,1,1,20241130,DET,PHI,22400301662,27890,0,1631311|1641737|1641741|1642272|1642348,2025


In [2]:
master['SHOT_ATTEMPTED_FLAG'].value_counts()
master['SHOT_MADE_FLAG'].value_counts()

0    27865
1    24035
Name: SHOT_MADE_FLAG, dtype: int64

In [3]:
shots = master.groupby(['year','SHOT_ZONE_RANGE','SHOT_DISTANCE']).sum(numeric_only=True)[['SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG']].reset_index()

shots.rename(columns={'SHOT_ATTEMPTED_FLAG':'FGA','SHOT_MADE_FLAG':'FGM'},inplace=True)
shots

,year,SHOT_ZONE_RANGE,SHOT_DISTANCE,FGA,FGM
0,2025,16-24 ft.,16,557,227
1,2025,16-24 ft.,17,496,213
2,2025,16-24 ft.,18,403,151
3,2025,16-24 ft.,19,343,136
4,2025,16-24 ft.,20,301,114
...,...,...,...,...,...
81,2025,Less Than 8 ft.,4,1846,796
82,2025,Less Than 8 ft.,5,1573,668
83,2025,Less Than 8 ft.,6,1349,540
84,2025,Less Than 8 ft.,7,1095,447


In [4]:
for year in range(start_year,2026):
    shot_distance=shots[shots.year==year].reset_index()
    shot_distance.drop(columns='year',inplace=True)
    shot_distance.to_csv(str(year)+'.csv',index=False)

In [5]:
master['SHOT_ZONE_RANGE'].unique()

array(['16-24 ft.', '8-16 ft.', 'Less Than 8 ft.', '24+ ft.',
       'Back Court Shot'], dtype=object)